<a href="https://colab.research.google.com/github/tannghia1234/ML231/blob/main/OT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO

path = r"D:\data\HBD21 dataset\Original_dataset\Sabotage_violence\sabotage_violence (47).mp4"
conf_threshold = 0.5
tracking_class = 0

tracker = DeepSort(max_age = 5)
model = YOLO("yolov8n.pt")
tracks = []
cap = cv2.VideoCapture(path)
while True:
    ret,frame = cap.read()
    if not ret:
        break
    results = model(frame)
    detect = []
    for data in results[0].boxes.data.tolist():
        confidence = data[4]

        if float(confidence) < conf_threshold:
            continue

        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])

        detect.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])
    tracks = tracker.update_tracks(detect,frame=frame)
    for track in tracks:
        if track.is_confirmed():
            track_id= track.track_id
            ltrb = track.to_ltrb()
            class_id = track.get_det_class()
            x1,y1,x2,y2 = map(int,ltrb)
            label = f"person-{track_id}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), GREEN, 2)
            cv2.rectangle(frame, (x1-1, y1 - 20), (x1 + len(label)*12, y1), GREEN, -1)
            cv2.putText(frame, label, (xmin + 5, ymin - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)
    cv2.imshow("Frame", frame)
    cv2.waitKey(0)
    if cv2.waitKey(1) == ord("q"):
        break

video_cap.release()
writer.release()
cv2.destroyAllWindows()






